# Build a Text Classifer
We'll apply Markov

A text classifier: takes an input, a string a text and predicts category it belongs to

ex: Is this Poem written by Robert Frost or Edgar Allan Poe?

Movie Reviews positive or negative, or spam detectors spem or not spam use the exact same code

#### Note: the goal of this exercise is not to build best most accurate classifier, otherwise we'd use pre-trained DL models, the goal is to practice.

#### Supervised or unsupervised?
- Text classification is example of supervised but markov models are unsupervised (training data is just text; no labels)
- so how do we use markov models in supervised context?
- answer: apply Bayes Rule (and build a Bayes classifier)
- p(y|x) = p(x|y)* p(y)/ p(x)

### Bayes Classifier:
- Poems by Frost: ==> Markov model: (A0, π0)
- Poems by Poe: ==> Markov model: (A1, π1)
- Train two separate models each has their own A matrices and π vectors.
- Remember Markov models allow to calculate probability of a sequence for a given A and π
- an unknown test poem: p(x|author = Frost) and p(x|author = Poe) hesapla
#### applying Bayes Rule:
- we obtained (poem|author) but we wanted (author|poem)
- if I have this distribution my prediction will be the highest of (poem|author)
- k^* = argmax p(class = k|x) x: poem, class k: author
- k-star is the argmax of probability classk given input x over all classes k.
- p(author|poem) = p(poem|author)* p(author) / p(poem)
- since we're taking argmax paydanın bi öenmi yok.
- if p(author) is uniform, we have no resason to believe one author to be more likely we can avoid that argument to
- hence : k-star = argmax log(p(poem|author = k))
#### this method is maximum likelihood because it's still a likelihood

## Exercise Prompt

- download dataset
- build a classifier that can distinguish between two authors
- compute train and test accuracy
- one class may be imbalanced so check for class imbalance, compute f1 score if imbalances

### Details
- loop through each data line and save each line to a list (one line == one sample)
- save labels (which line belongs to which poet, that'd be another list)
- train-test split
- create a mapping from unique word to unique integer index
- loop through data tokenize each line (string split should suffice)
- assign each unique word a unique index
- create a special index for unknown words
- convert each line to a list of index
- train a markov model for each class
- Smoothing and logs
- don't forget to compute the priors p(class = k) for bayes rule
- write a function to compute the posterior for each class given an input, it should take a sequence and compute the log-posterior for each class
- take argmax over each posterior
- make predictions for both train and test set
- compute accuracy for both train and test set
- check for the class imbalances (this info is in priors)
- if yes, compute confusion matrix to see which class has incorrect preds most often


In [1]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

--2023-06-10 13:01:00--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26622 (26K) [text/plain]
Saving to: 'edgar_allan_poe.txt'

edgar_allan_poe.txt 100%[===================>]  26.00K  --.-KB/s    in 0.001s  

2023-06-10 13:01:01 (19.0 MB/s) - 'edgar_allan_poe.txt' saved [26622/26622]

--2023-06-10 13:01:01--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP re

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

#dikkat et pandas indirmedik dataframe'lik bi durum yok.

In [4]:
!head edgar_allan_poe.txt

LO! Death hath rear'd himself a throne
In a strange city, all alone,
Far down within the dim west
Where the good, and the bad, and the worst, and the best,
Have gone to their eternal rest.
 
There shrines, and palaces, and towers
Are not like any thing of ours
Oh no! O no! ours never loom
To heaven with that ungodly gloom!


In [6]:
!head robert_frost.txt

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there


In [7]:
### hmmm kaç tane şiir olduğunu nerden bilcem, bi şiir nerde başlayıp nerde bitiyo?
#text document'ta nasıl istediğim kadar satır görürüm ? 

#çözüme bakmadan yapılcak gibi değil sanki...

## REMEMBER EACH LINE OF POEM WILL BE SAMPLE!!!
#google'a baktım
## File_object = open(r"File_Name","Access_Mode")
#access mode için "a" yazmış google'daki örnek: append mode'muş

In [11]:
RF = open(r'robert_frost.txt', "a")
EAP = open(r'edgar_allan_poe.txt', "a")


In [12]:
# readlines() : Reads all the lines and return them as each line a string element in a list.
 # File_object.readlines()

RF.readlines()

UnsupportedOperation: not readable

1- read() : Returns the read bytes in form of a string. Reads n bytes, if no n specified, reads the entire file.

File_object.read([n])

2 -readline() : Reads a line of the file and returns in form of a string.For specified n, reads at most n bytes. 

However, does not reads more than one line, even if n exceeds the length of the line.

File_object.readline([n])

readlines() : Reads all the lines and return them as each line a string element in a list.

  File_object.readlines()

In [24]:
### read mode'a çevirsek
RF = open(r'robert_frost.txt', "r+")
EAP = open(r'edgar_allan_poe.txt', "r+")
X_frost = RF.readlines()
len(X_frost)

1581

In [31]:
Y_frost = ["Frost" for i in range(1581)]
Y_frost

['Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',
 'Frost',


In [32]:
### OLMUYO PESSSS!!!! AYRICA BUNUN MARKOV MODELLE NE ALAKASI VAR ? 

In [36]:
# create a list that conains the names of our text files: 
input_files = [
  'edgar_allan_poe.txt',
  'robert_frost.txt',
]
#enumerate function
[print(label,f) for label, f in enumerate(input_files)]

0 edgar_allan_poe.txt
1 robert_frost.txt


[None, None]

In [37]:
open("edgar_allan_poe.txt")

<_io.TextIOWrapper name='edgar_allan_poe.txt' mode='r' encoding='UTF-8'>

In [38]:
#collect data into lists:
input_texts = []
labels = []

for label,f in enumerate(input_files):
    print(f"{f} corresponts to label {label}") #poe corresponds to label 0 and robert to label 1 olcak
    
    for line in open(f): #txt dosyasını açar and loop through each line of the file
        line = line.rstrip().lower() #lstrip, rstrip, strip Bu metodlar bir dizenin sol ve/veya sağ tarafındaki boş karakterleri kaldırır. 
        if line: #as recall some lines are empty, so this focus on nonempty lines, run only if nonempty
            #remove punctutation: understanding this code not crucial at thispoint
            line = line.translate(str.maketrans('', '', string.punctuation))

            input_texts.append(line) ## nonempty tüm line'ları o listeye ekled,
            labels.append(label) ## önce tüm poe'ları ve label'ları, sonra tüm frosları ve labelları
            
    

edgar_allan_poe.txt corresponts to label 0
robert_frost.txt corresponts to label 1


In [39]:
Text_train, Text_test, Y_train, Y_test = train_test_split(input_texts, labels) 
## acaba default parameterda kaçta kaçı test_size?

In [40]:
len(Y_train), len(Y_test)

(1615, 539)

In [42]:
#print a few rows from train_text
Text_train[:5]

['and experts said that deep down in the mountain',
 'reason enough there was no property',
 'of lipbegotten words',
 'the breath of air had died of cold',
 'i took the one less traveled by']

In [43]:
#next to convert our text to integers, create word2idx map

idx = 0
word2idx = {}

for line in Text_train:
    tokens = line.split()
    for token in tokens:
        if token not in word2idx:
            word2idx[token] = idx
            idx += 1


In [44]:
word2idx

{'and': 0,
 'experts': 1,
 'said': 2,
 'that': 3,
 'deep': 4,
 'down': 5,
 'in': 6,
 'the': 7,
 'mountain': 8,
 'reason': 9,
 'enough': 10,
 'there': 11,
 'was': 12,
 'no': 13,
 'property': 14,
 'of': 15,
 'lipbegotten': 16,
 'words': 17,
 'breath': 18,
 'air': 19,
 'had': 20,
 'died': 21,
 'cold': 22,
 'i': 23,
 'took': 24,
 'one': 25,
 'less': 26,
 'traveled': 27,
 'by': 28,
 'ten': 29,
 'million': 30,
 'silver': 31,
 'lizards': 32,
 'out': 33,
 'snow': 34,
 'italian': 35,
 'tones': 36,
 'made': 37,
 'only': 38,
 'to': 39,
 'be': 40,
 'murmured': 41,
 'all': 42,
 'is': 43,
 'if': 44,
 'id': 45,
 'aknown': 46,
 'when': 47,
 'young': 48,
 'doubt': 49,
 'its': 50,
 'grown': 51,
 'up': 52,
 'some': 53,
 'woods': 54,
 'around': 55,
 'it': 56,
 'then': 57,
 'other': 58,
 'as': 59,
 'just': 60,
 'fair': 61,
 'books': 62,
 'should': 63,
 'say': 64,
 'are': 65,
 'what': 66,
 'needed': 67,
 'could': 68,
 'have': 69,
 'told': 70,
 'them': 71,
 'any': 72,
 'time': 73,
 'this': 74,
 'twelvemonth'

In [45]:
len(word2idx)

2537

In [47]:
#convert data into integer format

train_text_int = []
test_text_int = []

for line in Text_train:
    tokens = line.split()
    line_as_int = [word2idx[token] for token in tokens] ## list comprehension
    train_text_int.append(line_as_int) ##benim düşümdüğümle aynı

for line in Text_test:
    tokens = line.split()
    line_as_int = [word2idx.get(token, 0) for token in tokens] #the only difference is this line
    #in case not every word in the test set not appear in dictionary, we use get function to get the value for that key
    #this function returns zero as default value (yalnız index zero "and" değil miydiii???? 
    #this corresponds to unknown tken dedi)
    test_text_int.append(line_as_int)

In [48]:
train_text_int[100:105]

[[39, 352, 56, 393, 3, 23, 12, 83, 394],
 [39, 395, 71, 72, 396, 397, 398],
 [399, 400, 207, 401, 23, 249],
 [402, 25, 403, 404, 405, 273, 406],
 [272, 407, 408, 409, 7, 227, 410, 15, 217, 411]]

In [49]:
## initialize A and π matrices for both classes
#since we have two classes we'll have two markov models: 2 A and 2π

V = len(word2idx) #equivalent to what we called M in theoric lectures, number of probabilities for a word to take

A0 = np.ones((V,V))
π0 = np.ones(V)

A1 = np.ones((V,V))
π1 = np.ones(V)

#the reason we initialized matrix and vectors of 1 but not zeroes bcz we'll use add-one smoothing

Sıradaki step'ten itibaren Markov modelin konuyla ne alaksı olduğunu anlıycaz galiba

In [81]:
#compute counts for A and π

##NOTE: BCZ A AND π ARE OBJECTS ANY MODIFICATION INSDIDE THE FUNCION WILL BE REMEMBERED OUTSIDE AS WELL

def compute_counts(text_as_int, A, π): #the function takes three argumens
    for tokens in text_as_int: #her bi token bi liste (line as int)
        last_idx= None
        for idx in tokens: #her line'daki kelimeler integer olarak
            if last_idx is None: 
                #when we're in the beginning of the sentence, then we'll apply initial distribution
                π[idx] += 1 #π is a vector
            else:
                #we're not in the first word
                A[last_idx, idx] += 1
            #update last idx:
            last_idx = idx
            
compute_counts([t for t, y in zip(train_text_int, Y_train) if y == 0], A0, π0) #poe
compute_counts([t for t, y in zip(train_text_int, Y_train) if y == 1], A1, π1) #frost

#t for (t,y) yani Y değeri 0 olan text'leri kullan - poe'nun text'lerini

    

In [82]:
#normalize A and π so they're valid probability matrices (HAH BNU diyodum)

#each row need to sum to one so divide A's rows buy their sums

A0 /= A0.sum(axis=1, keepdims = True)
π0 = π0 / (π0.sum())

A1 /= A1.sum(axis=1, keepdims = True)
π1 = π1 /(π1.sum())


In [71]:
π1

array([0.0263085 , 0.00027693, 0.00055386, ..., 0.00027693, 0.00027693,
       0.00027693])

In [72]:
π1.sum()

0.9999999999999999

In [83]:
#log A and π
logA0 = np.log(A0)
logπ0 = np.log(π0)

logA1 = np.log(A1)
logπ1 = np.log(π1)

In [84]:
# compute priors
count0 = sum(y == 0 for y in Y_train)
count1 = sum(y == 1 for y in Y_train)
total = len(Y_train)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.334984520123839, 0.6650154798761609)

- As we can see there are a lot more poe lines than frost lines,

- it's not a good idea to look maximum likelihood, we need to look at posteriors

- also since classes are imbalanced, use a metrics other than accuracy

#### here is why we use markov model instead of classifier from sklearn

In [85]:
#build a classifier:
# normally we build classifiers in a sklearn kind of interface with fit and predict method

#however, we've already done the fitting part by training A and π
#As an exercise you can also try with that method.

class Classifier:
    def __init__(self, logAs, logπs, logpriors):
        self.logAs = logAs
        self.logπs = logπs
        self.logpriors = logpriors
        self.K = len(logpriors) # number of classes
        
    def _compute_log_likelihood(self, input_, class_):
        #this will take as input a text, represent it by an integer and a class, an integer for each author
        logA = self.logAs[class_]
        logπ = self.logπs[class_]
        
        last_idx = None #initialize this for begging of sentence
        logprob = 0 #this will store the final answer
        for idx in input_: ### BÖYLE YAZABİLMEMİZİN SEBEBİ ÇOKTAN TÜM TEXT'LER LİSTE ÇEVRİLİP TOKENİZE EDİLDİ Mİ DİYE
              if last_idx is None:
        # it's the first token
                logprob += logπ[idx] #o indeksin başta gelmesi prob'unu ekledik
              else:
                logprob += logA[last_idx, idx] #o indeksin kendinden bi önceki kelmeyi takip etmesi probunu ekledik
      
      # update last_idx
              last_idx = idx
    
        return logprob  ### bu bize herhangi bi line'ın class'ta verilmiş author tarafından yazılma log-prob verir
    
    def predict(self, inputs): #input'lar ne? #bi şiir bütünü galiba çünkü şiirler tek line değil diye mi? 
        #inputs is a list of input sequences dedi ##Bİ SÜRÜ INPUTA AYNI ANDA TAHMIN YAPSIN DİYE Mİ
        predictions = np.zeros(len(inputs)) #begin by initializin an array
        for i, input_ in enumerate(inputs): 
            posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] for c in range(self.K)] ## posterior paydayı saymasa da bayes de bütün payı katar logpriors ihtimsli de
            pred = np.argmax(posteriors)
            predictions[i] = pred #o input'a ait prediction, pred olmuş olur
        return predictions


In [101]:
# each array must be in order since classes are assumed to index these lists
### We're creating an istance of our class
clf = Classifier([logA0, logA1], [logπ0, logπ1], [logp0, logp1])
clf.predict(train_text_int)

array([1., 1., 0., ..., 0., 1., 1.])

In [106]:
P_train = clf.predict(train_text_int)
print(f"Train acc: {np.mean(P_train == Y_train, dtype=np.float64)}")


Train acc: 1.0


In [108]:
P_test = clf.predict(test_text_int)
print(f"Test acc: {np.mean(P_test == Y_test)}") #much lower

Test acc: 0.7421150278293135


Hocanınkinde farklı cevaplar çıktı, ama test ve train split'i aynı random seed ile yapmadığımız için muhtemelen

hocada test set 0.82 çıktı

In [109]:
from sklearn.metrics import confusion_matrix, f1_score
# read about F-score: https://en.wikipedia.org/wiki/F-score

In [110]:
cm = confusion_matrix(Y_train, P_train)
cm

array([[ 541,    0],
       [   0, 1074]])

In [111]:
cm_test = confusion_matrix(Y_test, P_test)
cm_test

array([[136,  41],
       [ 98, 264]])

In [113]:
f1_score(Y_train, P_train)

1.0

In [114]:
f1_score(Y_test, P_test)

0.7916041979010495

In [ ]:
### WHAT IS F-1 SCORE WHEN WE USE THEM?